# How to write multi-batch `BatchRequest` - `ConfiguredAsset` Example
* A `BatchRequest` facilitates the return of a `batch` of data from a configured `Datasource`. To find more about `Batches`, please refer to the [related documentation](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_get_a_batch_of_data_from_a_configured_datasource#1-construct-a-batchrequest). 
* A `BatchRequest` can return 0 or more Batches of data depending on the underlying data, and how it is configured. This guide will help you configure `BatchRequests` to return multiple batches, which can be used by
   1. Self-Initializing Expectations to estimate parameters
   2. DataAssistants to profile your data and create and Expectation suite self-intialized parameters.
   
* Note : Multi-batch BatchRequests are not supported in `RuntimeDataConnector`.

## FileSystem Example

### Example Directory

Imagine we have a directory of 12 csv files, each corresponding to 1 month of Taxi rider data

```
yellow_tripdata_sample_2020-01.csv
yellow_tripdata_sample_2020-02.csv
yellow_tripdata_sample_2020-03.csv
yellow_tripdata_sample_2020-04.csv
yellow_tripdata_sample_2020-05.csv
yellow_tripdata_sample_2020-06.csv
yellow_tripdata_sample_2020-07.csv
yellow_tripdata_sample_2020-08.csv
yellow_tripdata_sample_2020-09.csv
yellow_tripdata_sample_2020-10.csv
yellow_tripdata_sample_2020-11.csv
yellow_tripdata_sample_2020-12.csv
```


In [1]:
import great_expectations as ge

from ruamel import yaml

from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest

from great_expectations.rule_based_profiler.rule.rule import Rule
from great_expectations.rule_based_profiler.rule_based_profiler import RuleBasedProfiler, RuleBasedProfilerResult

from great_expectations.rule_based_profiler.domain_builder import (
    DomainBuilder,
    ColumnDomainBuilder,
)
from great_expectations.rule_based_profiler.parameter_builder import (
    MetricMultiBatchParameterBuilder,
)
from great_expectations.rule_based_profiler.expectation_configuration_builder import (
    DefaultExpectationConfigurationBuilder,
)

* Load `DataContext`

In [2]:
data_context: ge.DataContext = ge.get_context()

### `ConfiguredDataConnector` Example

* Add `Datasource` named `taxi_multi_batch_configured_datasource` with two `ConfiguredAssetDataConnectors `. A key difference is in the `pattern` used to identify one Batch from another. 

* The first DataConnector is called `configured_data_connector_single_batch_asset`, which names two `assets`: `yellow_trip_data_jan` and `yellow_trip_data_feb`. 
    * This can be seen in the output of `test_yaml_config()`, which shows the 2 data assets, with 1 Batch each
    * Here is the output: 
    ```	
    Available data_asset_names (2 of 2):
		yellow_trip_data_feb (1 of 1): ['yellow_tripdata_sample_2020-02.csv']
		yellow_trip_data_jan (1 of 1): ['yellow_tripdata_sample_2020-01.csv']
    ```
    * **Note**: in this case we actually don't need a `group_name` defined if we are just saying our pattern was `yellow_tripdata_sample_2020-(01)\\.csv`. However, GE currently doesn't allow a `pattern` to be defined without `group_name` also defined. So in our case, we set regex defined in `pattern` to capture the `month`.

* A second DataConnector is called `configured_data_connector_multi_batch_asset`:
    * It defines one asset `yellow_tripdata_all_months`, that takes all 12 `yellow_tripdata_sample_2020` files and maps it to the same asset, with each of the 12 months corresponding to Batches for the asset. 
    * In order to do this, we define the `pattern` as `"yellow_tripdata_sample_(.*)\\.csv"` which matches all months, with one capture group `(.*)` defined as `month`. This allows the resulting Batches to be distinguishable from one another, and allows them be specified using a `batch_definition`.
    * The results can be seen in the output of `test_yaml_config()`, which shows 3 of the 12 Batches corresponding to `yellow_tripdata_all_months`
    * Here is the output:
 ```
 Available data_asset_names (1 of 1):
       yellow_tripdata_all_months (3 of 12): ['yellow_tripdata_sample_2020-01.csv', 'yellow_tripdata_sample_2020-02.csv', 'yellow_tripdata_sample_2020-03.csv']
 ```

In [32]:
data_path: str = "../../../../test_sets/taxi_yellow_tripdata_samples/samples_2020"

datasource_config = {
    "name": "taxi_multi_batch_configured_datasource",
    "class_name": "Datasource",
    "module_name": "great_expectations.datasource",
    "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine",
    },
    "data_connectors": {
        "configured_data_connector_single_batch_asset": {
            "class_name": "ConfiguredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "assets": {
                "yellow_trip_data_jan":
                {
                    "group_names": ["month"],
                    "pattern": "yellow_tripdata_sample_2020-(01)\\.csv",
                },
                "yellow_trip_data_feb":
                {
                    "group_names": ["month"],
                    "pattern": "yellow_tripdata_sample_2020-(02)\\.csv",
                }
            },
        },
        
        "configured_data_connector_multi_batch_asset": {
            "class_name": "ConfiguredAssetFilesystemDataConnector",
            "base_directory": data_path,
            "assets": {
                "yellow_tripdata_all_months":{
                    "pattern": "yellow_tripdata_sample_(.*)\\.csv",
                    "group_names": ["month"],

                }
            },
        },
        
    },
}

data_context.test_yaml_config(yaml.dump(datasource_config))

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	configured_data_connector_multi_batch_asset : ConfiguredAssetFilesystemDataConnector

	Available data_asset_names (1 of 1):
		yellow_tripdata_all_months (3 of 12): ['yellow_tripdata_sample_2020-01.csv', 'yellow_tripdata_sample_2020-02.csv', 'yellow_tripdata_sample_2020-03.csv']

	Unmatched data_references (0 of 0):[]

	configured_data_connector_single_batch_asset : ConfiguredAssetFilesystemDataConnector

	Available data_asset_names (2 of 2):
		yellow_trip_data_feb (1 of 1): ['yellow_tripdata_sample_2020-02.csv']
		yellow_trip_data_jan (1 of 1): ['yellow_tripdata_sample_2020-01.csv']

	Unmatched data_references (3 of 22):['yellow_tripdata_sample_2020-01.csv', 'yellow_tripdata_sample_2020-03.csv', 'yellow_tripdata_sample_2020-04.csv']



In [33]:
# add_datasource only if it doesn't already exist in our configuration
try:
    data_context.get_datasource(datasource_config["name"])
except ValueError:
    data_context.add_datasource(**datasource_config)

## BatchRequest

* Depending on which `DataConnector` you send a `BatchRequest` to, you will retrieve a different number of `Batches`

* Single Batch returned by `configured_data_connector_single_batch_asset` DataConnector.

In [40]:
single_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_single_batch_asset",
    data_asset_name="yellow_trip_data_jan",
)

In [41]:
batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request)

In [42]:
batch_list

* Multi Batch returned by the `configured_data_connector_multi_batch_asset` DataConnector.

In [43]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_all_months",
)

In [44]:
multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)

In [45]:
multi_batch_batch_list

* You can also get a single Batch from a multi-batch DataConnector by passing in `data_connector_query`. Index `-1` will return the most recent (month = `12`) batch. 

In [47]:
single_batch_batch_request_from_multi: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_configured_datasource",
    data_connector_name="configured_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_all_months",
    data_connector_query={
        "index": -1 # month: 12
    }
)

In [48]:
batch_list = data_context.get_batch_list(batch_request=single_batch_batch_request_from_multi)

In [49]:
batch_list

In [51]:
batch_list[0].to_dict() # 'batch_identifiers': {'month': '12'}},

{'data': '<great_expectations.execution_engine.pandas_batch_data.PandasBatchData object at 0x7f97f1d1d040>',
 'batch_request': {'datasource_name': 'taxi_multi_batch_configured_datasource',
  'data_connector_name': 'configured_data_connector_multi_batch_asset',
  'data_asset_name': 'yellow_tripdata_all_months',
  'limit': None,
  'data_connector_query': {'index': -1},
  'batch_spec_passthrough': None},
 'batch_definition': {'datasource_name': 'taxi_multi_batch_configured_datasource',
  'data_connector_name': 'configured_data_connector_multi_batch_asset',
  'data_asset_name': 'yellow_tripdata_all_months',
  'batch_identifiers': {'month': '2020-12'}},
 'batch_spec': {'path': '/Users/work/Development/great_expectations/tests/test_fixtures/rule_based_profiler/example_notebooks/great_expectations/../../../../test_sets/taxi_yellow_tripdata_samples/samples_2020/yellow_tripdata_sample_2020-12.csv'},
 'batch_markers': {'ge_load_time': '20220722T031635.271527Z',
  'pandas_data_fingerprint': 'b1b9

# Using auto-initializing `Expectations` to generate parameters

* We will generate a `Validator` using our `multi_batch_batch_list`

In [52]:
multi_batch_batch_list = data_context.get_batch_list(batch_request=multi_batch_batch_request)

In [53]:
example_suite = data_context.create_expectation_suite(expectation_suite_name="example_configured_suite", overwrite_existing=True)

In [54]:
validator = data_context.get_validator_using_batch_list(batch_list=multi_batch_batch_list, expectation_suite=example_suite)

* When you run methods on the validator, it will typically run on the most recent batch (index `-1`), even if the Validator has access to a longer Batch list. For example, notice that the `pickup_datetime` and `dropoff_datetime` below are all associated with December, indicating that it is with the most recent Batch.

In [55]:
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,2.0,2020-12-15 12:20:27,2020-12-15 12:40:49,4.0,5.76,1.0,N,209,237,1.0,21.0,0.0,0.5,2.50,0.0,0.3,26.80,2.5
1,2.0,2020-12-28 12:51:25,2020-12-28 13:15:12,1.0,11.64,1.0,N,161,220,1.0,33.5,0.0,0.5,5.00,2.8,0.3,44.60,2.5
2,2.0,2020-12-27 10:43:42,2020-12-27 10:51:05,1.0,1.22,1.0,N,163,48,1.0,7.0,0.0,0.5,2.06,0.0,0.3,12.36,2.5
3,2.0,2020-12-08 13:42:52,2020-12-08 13:54:45,1.0,1.84,1.0,N,137,229,2.0,9.0,0.0,0.5,0.00,0.0,0.3,12.30,2.5
4,2.0,2020-12-19 11:56:43,2020-12-19 12:08:43,1.0,1.55,1.0,N,24,74,1.0,9.5,0.0,0.5,2.58,0.0,0.3,12.88,0.0


### Typical Workflow
* A `batch_list` becomes really useful when you are calculating parameters for auto-initializing Expectations, as they us a `RuleBasedProfiler` under-the-hood to calculate parameters.

* Here is an example running `expect_column_median_to_be_between()` by "guessing" at the `min_value` and `max_value`. 

In [56]:
validator.expect_column_median_to_be_between(column="trip_distance", min_value=0, max_value=1)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "meta": {},
  "success": false,
  "result": {
    "observed_value": 1.61
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

* The observed value for the most recent batch (December/2020) is going to be `1.61`, which means the Expectation fails

* Now we run the same expectation again, but this time with `auto=True`. This means the `median` values are going to calculated across the `batch_list` associated with the `Validator` (ie 12 Batches for 2020), which gives the min value of `1.6` and the max value of `1.92`

In [57]:
validator.expect_column_median_to_be_between(column="trip_distance", auto=True)

Generating Expectations:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "meta": {},
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_median_to_be_between",
    "kwargs": {
      "strict_max": false,
      "column": "trip_distance",
      "max_value": 1.92,
      "strict_min": false,
      "min_value": 1.6
    },
    "meta": {
      "auto_generated_at": "20220722T031719.345725Z",
      "great_expectations_version": "0.15.13+79.g5c6db848e.dirty"
    }
  },
  "result": {
    "observed_value": 1.61
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

* The `auto=True` will also automatically run the Expectation against the most recent Batch (which has an observed value of `1.61`) and the Expectation will pass. 

* You can now save the `ExpectationSuite`.

In [58]:
validator.save_expectation_suite()

### Running the `ExpectationSuite` against single `Batch`

Now the ExpectationSuite can be used to validate single batches using a Checkpoint. As before, we can use `data_connector_query` to specify the batch that we would like to run the `Checkpoint` on, but the recommended way would be to use the `batch_identifier` parameter, where we have set `month` to `01` to specify the January 2020 batch.

In [61]:
multi_batch_batch_request: BatchRequest = BatchRequest(
    datasource_name="taxi_multi_batch_inferred_datasource",
    data_connector_name="inferred_data_connector_multi_batch_asset",
    data_asset_name="yellow_tripdata_sample_2020",
    #data_connector_query={
    #    "index": 0 # this one will correspond to Jan 2020
    #}
)


In [60]:
checkpoint_config = {
    "name": "my_checkpoint",
    "config_version": 1,
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": single_batch_batch_request_from_multi,
            "expectation_suite_name": "example_configured_suite",
            "batch_identifiers":{"month": "01"} # batch_identifier month is set to 01
        }
    ],
}
data_context.add_checkpoint(**checkpoint_config)

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "my_checkpoint",
  "profilers": [],
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "taxi_multi_batch_configured_datasource",
        "data_connector_name": "configured_data_connector_multi_batch_asset",
        "data_asset_name": "yellow_tripdata_all_months",
        "data_connector_query": {
          "index": 0
        }
      },

In [28]:
results = data_context.run_checkpoint(
    checkpoint_name="my_checkpoint"
)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
results.success

True